In [1]:
import numpy as np 
import cv2 
import glob

criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

objp = np.zeros((6*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:6].T.reshape(-1, 2)

objpoints = []
imgpoints = []

images_name_list = glob.glob("assets/left/left*.jpg")

for im_name in images_name_list:
    # print(im_name)
    img = cv2.imread(im_name) 
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ret, corners = cv2.findChessboardCorners(gray, (7,6), None)
    
    if ret == True:
        objpoints.append(objp)
        
        corners2 = cv2.cornerSubPix(gray, corners, (11,11), (-1, -1), criteria)
        imgpoints.append(corners)
        
        cv2.drawChessboardCorners(img, (7,6), corners2, ret)
        cv2.imshow("image", img)
        cv2.waitKey(100)


############ 相机标定
# 3D点和图像点
# cv2.calibrateCamera 相机标定
# 返回 camera matrix, distortion coefficients, rotation and translation vectors etc.
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("camera matrix:", mtx)
print("distortion", dist)

############## 保存参数
np.savez("left.npz", mtx=mtx, dist=dist,rvecs=rvecs,tvecs=tvecs )


############# 去失真
## OpenCV 有两种方法，
## 我们可以使用cv2.getOptimalNewCameraMatrix()基于自由缩放参数来细化相机矩阵。
## 如果缩放参数alpha=0，它将返回未扭曲的图像，其中包含最少的不需要像素。因此，它甚至可以删除图像角落的一些像素。
## 如果alpha=1，所有像素都保留一些额外的黑色图像。此函数还返回一个图像ROI，可用于裁剪结果。
imgtest = cv2.imread("assets/left/left12.jpg")
h, w = imgtest.shape[0:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))

cv2.imshow("left12", imgtest)

#### 方法一： cv2.undistort()，方法一的效率更高一些
# undistort
dst = cv2.undistort(imgtest, mtx, dist, None, newcameramtx)

# crop image
x, y, w, h = roi 
dst = dst[y:y+h, x:x+w]

cv2.imshow("left12-undistort", dst)
#### 方法二： using remapping
# undistort
mapx, mapy = cv2.initUndistortRectifyMap(mtx, dist, None, newcameramtx, (w,h), 5)
dst2 = cv2.remap(imgtest, mapx, mapy, cv2.INTER_LINEAR)

# crop the image
x, y, w, h = roi
dst2 = dst[y:y+h, x:x+w]
cv2.imshow("left12-remapping", dst)

cv2.waitKey(0)
cv2.destroyAllWindows()

# 反向投影计算误差
# print(objpoints)
mean_error = 0
for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], mtx, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2)/len(imgpoints2)
    mean_error += error
print( "total error: {}".format(mean_error/len(objpoints)) )


camera matrix: [[534.07088364   0.         341.53407554]
 [  0.         534.11914595 232.94565259]
 [  0.           0.           1.        ]]
distortion [[-2.92971637e-01  1.07706962e-01  1.31038376e-03 -3.11018780e-05
   4.34798110e-02]]
total error: 0.023686000375385676


In [4]:
npzfile = np.load("left.npz")
sorted(npzfile.files)


['dist', 'mtx', 'rvecs', 'tvecs']